## Analysis Challenge Assignment 1

Group member: Jingxuan Liao, Ruoqi(Mark) Wang, Yuchen(Rebecca) Wu 

* Data Wrangling Order: 
1. Safety(Select) 
    * Crime: filter the top 50th percentile in overall crime. 
2. Urban(Select) 
    * Metropolitan 
3. Quality(Select) 
    * Average Faculty Salary 
    * Percentage of full-time Faculty 
4. Diversity(Exclusion) 
    * Flag for specific race-only school 
    * Flag for men-only college/ Flag for women-only college


In [74]:
import pandas as pd
import numpy as np

In [75]:
Score_card = pd.read_csv("CollegeScorecard.csv")
Crime_rate = pd.read_csv("Crime_2015.csv")

/Users/mark/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We want to combine the two data sets by Cities, so we need to check if the columns names are identical.

In [62]:
print(Score_card.columns [: 20])
print(Crime_rate.columns [:20])

Index(['UNITID', 'OPEID', 'opeid6', 'INSTNM', 'CITY', 'STABBR', 'ZIP',
       'AccredAgency', 'INSTURL', 'NPCURL', 'sch_deg', 'HCM2', 'main',
       'NUMBRANCH', 'PREDDEG', 'HIGHDEG', 'CONTROL', 'st_fips', 'region',
       'LOCALE'],
      dtype='object')
Index(['MSA', 'ViolentCrime', 'Murder', 'Rape', 'Robbery', 'AggravatedAssault',
       'PropertyCrime', 'Burglary', 'Theft', 'MotorVehicleTheft', 'State',
       'City'],
      dtype='object')


We can see  that two columns names in different data sets are not identical, so we need to rename one of it. 

In [63]:
Crime_rate = Crime_rate.rename ({'City':'CITY'}, axis = 1)

### Data cleaning process: 
* firstly, we want to combine two data sets 
* Secondly, we need to extract the information we will analysis


In [64]:
mdf = pd.merge(Crime_rate, Score_card, how='right', on='CITY')
mdf.shape

(8687, 1740)

We have combined the two data sets. There are 8687 rows (institutions), and 1740 rows.

Now we want to extract the information we want. We are going to filter institutions by city size, safety of the city, diversity of the school, and quality of the school. 

#### Let's organize the data first. 

Firstly, We are going to analyze the safety of the city by the number of murder, rape, Robbery, Aggravated Assult, and MotorVehicleTheft. Thus, we can create a column by combining all of them. 

In [65]:
mdf['Crime_total'] = mdf["Murder"]+mdf['Rape']+mdf['Robbery']+mdf['AggravatedAssault']+mdf['MotorVehicleTheft']

Secondly, we are going to analyze the diversity of the school by figuring out if the school has restriction of race.  We discard all the institutions that have religious affiliations, and we only keep the institutions that do not have religious affiliations in our list. Also, we are excluding institutions that consist of predominantly one race, e.g. Flag for Historically Black College and University. Thus, we are going to combine the columns about the school stypes which have restriction on race.  

In [66]:
mdf['Race_score'] = mdf['HBCU'] + mdf['PBI'] + mdf['ANNHI'] + mdf['TRIBAL'] + mdf['AANAPII'] \
+ mdf['HSI'] + mdf['NANTI'] + mdf['MENONLY']+mdf['WOMENONLY']

Now, we want to extract the data which is useful for our filtering. All our judege is based on Local, Crime_total, average faculty salary, Percentage of full time Faculty, and races, so we extract those first. 

In [67]:
newmdf = mdf.filter(['INSTNM', 'CITY','Crime_total','LOCALE','AVGFACSAL', 'PFTFAC', 'Race_score', 'RELAFFIL'], axis=1)

Now, the new dataset(newmdf) contains the types of information that we want to analyze. 

In [68]:
newmdf.shape

(8687, 8)

The number of rows is still 8687, but the columns are 8 now. All those 8 columns are the types of information we are going to use to filter. 

#### Data filtering:

Our rubric to filter school is following: 

* Safety:

In order to address Alex’s needs of safety, which is reflected by low crime rate, we ordered all the categories in the crime_2015 dataset, and we only keep the institutions in which the cities are among the lower half of all cities (50%) in all of the crime rate categories.

* Urban:

In order to address Alex’s expectations to live a big city life, we use the index of “locale of institution”, and we only keep the institutions who are categorized as “City: Large (population of 250,000 or more)” in our list.

* Diversity:

Race: Flag for Historically Black College and University, predominantly black institution, Flag for Alaska Native Native Hawaiian serving institution, tribal college and university, Asian American Native American Pacific Islander-serving institution, Hispanic-serving institution, and Native American non-tribal institution - we would like to exclude the scholl with the value of "1".

Gender: Flag for men-only college/Flag for women-only college - exclude school with either value of “1” (Yes)


* Quality:

Faculty
Average Faculty salary - we are only keeping the institutions who are in the higher half among all institutions regarding average faculty salary - we regard this as a reflection of the institutions’ economic strength
Proportion of faculty that is full-time - we are only keeping stitutions that are in the higher half of the list regarding proportion of full time faculty - since we think this is a reflection of the institution’s ability to attract faculty, and the stability of the staff


Now, we need to see the descriptive of the data set in order to set limits to filter. 

In [69]:
newmdf_describe = newmdf.describe(include = 'all')
newmdf_describe

,INSTNM,CITY,Crime_total,LOCALE,AVGFACSAL,PFTFAC,Race_score,RELAFFIL
count,8687,8687,4211.000000,8213.000000,5194.000000,4596.000000,8216.000000,1036.000000
unique,7634,2545,NaN,NaN,NaN,NaN,NaN,NaN
top,Marinello School of Beauty,Springfield,NaN,NaN,NaN,NaN,NaN,NaN
freq,14,172,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,644.002968,19.012541,6004.462457,0.566776,0.060370,54.000965
std,NaN,NaN,271.460422,9.237607,2259.112032,0.311286,0.242742,20.905192
min,NaN,NaN,104.500000,11.000000,269.000000,0.000000,0.000000,22.000000
25%,NaN,NaN,449.700000,11.000000,4452.500000,0.295000,0.000000,30.000000
50%,NaN,NaN,614.100000,13.000000,5792.000000,0.535700,0.000000,54.000000
75%,NaN,NaN,821.100000,21.000000,7231.500000,0.875000,0.000000,71.000000


We now need to define some values with name because this is a way to make the code reproducible. 

In [70]:
Crime_total_mean = newmdf_describe.loc['mean', 'Crime_total']
AVGFACSAL_mean = newmdf_describe.loc['mean', 'AVGFACSAL']
PFTFAV_mean = newmdf_describe.loc['mean', 'PFTFAC']
Race_score_mean = newmdf_describe.loc['mean', 'Race_score']
RELAFFIL_mean = newmdf_describe.loc['mean', 'RELAFFIL']

We want to get targeted schools for Alex by our rubric. 

In [71]:
target_places = (newmdf['Crime_total'] < Crime_total_mean) & (newmdf['LOCALE'] == 11) \
& (newmdf['AVGFACSAL'] > AVGFACSAL_mean) & (newmdf['Race_score'] == 0) & (newmdf['PFTFAC'] > PFTFAV_mean) \
& (newmdf["RELAFFIL"] > RELAFFIL_mean)

We can use Race_score == 0 instead of Race_score_mean because according to the data dictionary, we only take the values = 0 under the race category we defined. Thus, the sum of 0 is 0. 

In [72]:
newmdf[target_places].shape

(20, 8)

Now the number schools has been narrowed down to 20 from 8687. 

In [73]:
newmdf[target_places]

,INSTNM,CITY,Crime_total,LOCALE,AVGFACSAL,PFTFAC,Race_score,RELAFFIL
521,Point Loma Nazarene University,San Diego,631.8,11.0,9653.0,1.0000,0.0,59.0
771,American University,Washington,485.5,11.0,11074.0,0.5764,0.0,71.0
772,American University,Washington,542.3,11.0,11074.0,0.5764,0.0,71.0
796,Wesley Theological Seminary,Washington,485.5,11.0,8700.0,0.9583,0.0,71.0
797,Wesley Theological Seminary,Washington,542.3,11.0,8700.0,0.9583,0.0,71.0
1148,Chicago Theological Seminary,Chicago,542.4,11.0,7725.0,1.0000,0.0,76.0
1149,Chicago Theological Seminary,Chicago,614.1,11.0,7725.0,1.0000,0.0,76.0
1265,McCormick Theological Seminary,Chicago,542.4,11.0,6654.0,1.0000,0.0,66.0
1266,McCormick Theological Seminary,Chicago,614.1,11.0,6654.0,1.0000,0.0,66.0
1672,Transylvania University,Lexington,506.6,11.0,8282.0,0.8362,0.0,61.0
